In [1]:
import datetime
import os
from tensorboardX import SummaryWriter

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pylab as plt

import torchvision.datasets as datasets
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = datasets.MNIST(
    root="../chap13/data", train=True, transform=transform, download=True)

test_dataset = datasets.MNIST(
    root="../chap13/data", train=False, transform=transform, download=True)

train_loader = DataLoader(
    train_dataset, batch_size=100, shuffle=True, num_workers=4, pin_memory=False)

test_loader = DataLoader(
    test_dataset, batch_size=100, shuffle=False, num_workers=4)

In [3]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.input1 = nn.Linear(input_dim, hidden_dim)
        self.input2 = nn.Linear(hidden_dim, hidden_dim)
        self.mean = nn.Linear(hidden_dim, latent_dim)
        self.var = nn.Linear(hidden_dim, latent_dim)

        self.LeakyReLU = nn.LeakyReLU(0.2)
        self.training = True

    def forward(self, x):
        h_ = self.LeakyReLU(self.input1(x))
        h_ = self.LeakyReLU(self.input2(h_))
        mean = self.mean(h_)
        log_var = self.var(h_)
        return mean, log_var

In [4]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.hidden1 = nn.Linear(latent_dim, hidden_dim)
        self.hidden2 = nn.Linear(hidden_dim, hidden_dim)
        self.output = nn.Linear(hidden_dim, output_dim)
        self.LeakyReLU = nn.LeakyReLU(0.2)

    def forward(self, x):
        h = self.LeakyReLU(self.hidden1(x))
        h = self.LeakyReLU(self.hidden2(h))
        x_hat = torch.sigmoid(self.output(h))
        return x_hat

In [5]:
class Model(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Model, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder

    def reparameterization(self, mean, var):
        epsilon = torch.randn_like(var).to(device)
        z = mean + var*epsilon
        return z

    def forward(self, x):
        mean, log_var = self.Encoder(x)
        z = self.reparameterization(mean, torch.exp(0.5 * log_var))
        x_hat = self.Decoder(z)
        return x_hat, mean, log_var

In [6]:
x_dim = 784
hidden_dim = 400
latent_dim = 200
epochs = 30
batch_size = 100

encoder = Encoder(input_dim=x_dim, hidden_dim=hidden_dim,
                  latent_dim=latent_dim)
decoder = Decoder(latent_dim=latent_dim,
                  hidden_dim=hidden_dim, output_dim=x_dim)

model = Model(Encoder=encoder, Decoder=decoder).to(device)

In [7]:
def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.binary_cross_entropy(
        x_hat, x, reduction='sum')
    KLD = - 0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
    return reproduction_loss, KLD


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
saved_loc = 'scalar/'
writer = SummaryWriter(saved_loc)

model.train()


def train(epoch, model, train_loader, optimizer):
    train_loss = 0
    for batch_idx, (x, _) in enumerate(train_loader):
        x = x.view(batch_size, x_dim)
        x = x.to(device)

        optimizer.zero_grad()
        x_hat, mean, log_var = model(x)
        BCE, KLD = loss_function(x, x_hat, mean, log_var)
        loss = BCE + KLD
        writer.add_scalar("Train/Reconstruction Error", BCE.item(),
                          batch_idx + epoch * (len(train_loader.dataset)/batch_size))
        writer.add_scalar("Train/KL-Divergence", KLD.item(),
                          batch_idx + epoch * (len(train_loader.dataset)/batch_size))
        writer.add_scalar("Train/Total Loss", loss.item(), batch_idx +
                          epoch * (len(train_loader.dataset)/batch_size))

        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                epoch, batch_idx * len(x), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(x)))

    print("======> Epoch: {} Average loss: {:.4f}".format(
        epoch, train_loss / len(train_loader.dataset)))

In [9]:
def test(epoch, model, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for batch_idx, (x, _) in enumerate(test_loader):
            x = x.view(batch_size, x_dim)
            x = x.to(device)
            x_hat, mean, log_var = model(x)
            BCE, KLD = loss_function(x, x_hat, mean, log_var)
            loss = BCE + KLD

            writer.add_scalar("Test/Reconstruction Error", BCE.item(),
                              batch_idx + epoch * (len(test_loader.dataset)/batch_size))
            writer.add_scalar("Test/KL-Divergence", KLD.item(),
                              batch_idx + epoch * (len(test_loader.dataset)/batch_size))
            writer.add_scalar("Test/Total Loss", loss.item(), batch_idx +
                              epoch * (len(test_loader.dataset)/batch_size))
            test_loss += loss.item()

            if batch_idx == 0:
                n = min(x.size(0), 8)
                comparison = torch.cat(
                    [x[:n], x_hat.view(batch_size, x_dim)[:n]])
                grid = torchvision.utils.make_grid(comparison.cpu())
                writer.add_image(
                    "Test image - Above: Real data, below: reconstruction data", grid, epoch)

In [10]:
from tqdm.auto import tqdm
for epoch in tqdm(range(0, epochs)):
    train(epoch, model, train_loader, optimizer)
    test(epoch, model, test_loader)
    print("\n")
writer.close()

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/30 [00:00<?, ?it/s]

Train Epoch: 0 [0/60000 (0%)]	 Loss: 545.139258
Train Epoch: 0 [10000/60000 (17%)]	 Loss: 198.182793
Train Epoch: 0 [20000/60000 (33%)]	 Loss: 180.591875
Train Epoch: 0 [30000/60000 (50%)]	 Loss: 168.394277
Train Epoch: 0 [40000/60000 (67%)]	 Loss: 150.181865
Train Epoch: 0 [50000/60000 (83%)]	 Loss: 148.615000
======> Epoch: 0 Average loss: 173.7764


  3%|▎         | 1/30 [00:01<00:32,  1.12s/it]



Train Epoch: 1 [0/60000 (0%)]	 Loss: 146.846182
Train Epoch: 1 [10000/60000 (17%)]	 Loss: 134.781787
Train Epoch: 1 [20000/60000 (33%)]	 Loss: 123.950195
Train Epoch: 1 [30000/60000 (50%)]	 Loss: 127.748301
Train Epoch: 1 [40000/60000 (67%)]	 Loss: 129.141113
Train Epoch: 1 [50000/60000 (83%)]	 Loss: 122.684102
======> Epoch: 1 Average loss: 128.6877


  7%|▋         | 2/30 [00:02<00:30,  1.10s/it]

...

 93%|█████████▎| 28/30 [00:29<00:02,  1.05s/it]



Train Epoch: 28 [0/60000 (0%)]	 Loss: 96.874551
Train Epoch: 28 [10000/60000 (17%)]	 Loss: 101.833477
Train Epoch: 28 [20000/60000 (33%)]	 Loss: 95.250479
Train Epoch: 28 [30000/60000 (50%)]	 Loss: 99.681719
Train Epoch: 28 [40000/60000 (67%)]	 Loss: 100.682402
Train Epoch: 28 [50000/60000 (83%)]	 Loss: 98.413037
======> Epoch: 28 Average loss: 100.2927


 97%|█████████▋| 29/30 [00:30<00:01,  1.07s/it]



Train Epoch: 29 [0/60000 (0%)]	 Loss: 101.295098
Train Epoch: 29 [10000/60000 (17%)]	 Loss: 102.861465
Train Epoch: 29 [20000/60000 (33%)]	 Loss: 97.502266
Train Epoch: 29 [30000/60000 (50%)]	 Loss: 101.330381
Train Epoch: 29 [40000/60000 (67%)]	 Loss: 97.677188
Train Epoch: 29 [50000/60000 (83%)]	 Loss: 101.826963
======> Epoch: 29 Average loss: 100.2407


100%|██████████| 30/30 [00:31<00:00,  1.06s/it]

In [11]:
%load_ext tensorboard
%tensorboard --logdir scalar --port=9000

ERROR: Failed to launch TensorBoard (exited with 255).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

E1010 02:03:54.039050 139957028296512 program.py:300] TensorBoard could not bind to port 9000, it was already in use
ERROR: TensorBoard could not bind to port 9000, it was already in use